Import libraries.

In [114]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import errno

Get list of funds registered in Austria and list of Meldefonds from [__Profitweb__](https://www.profitweb.at/).

Save the files as _.xlsx_ to be able to read them in.

In [115]:
oekb_rfs_file = pd.read_excel('data/oekb_rfs.xlsx')
oekb_melde_file = pd.read_excel('data/oekb_melde.xlsx')

[__Generate RFS Report__](http://lctprod2.int.thomsonreuters.com/reportingTool/Report?id=76) and read it in as _.csv_.

__Domicile__: blank

__RFS__: blank

__Asset Universe__: Closed End Funds, Exchange Traded Funds, Hedge Funds, Insurace Funds, Mutual Funds, Pension Funds, Investment Trusts

__Status__: Active, Created, Pending, Suspended, Withdrawn

In [116]:
report_file = pd.read_csv('data/Report_RFSSwitzerlandandRFSAustriaReport_.csv')

C:\Users\U6046944\AppData\Local\Continuum\anaconda3\envs\python-for-data-science\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (9,10,15,19,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


---

Create sets of OeKB RFS and Meldefonds. Remove Austrian funds from Meldefonds list.

In [117]:
# OeKB Funds with RFS Austria
oekb_rfs = set(oekb_rfs_file['ISIN'])

#OeKB Meldefonds and Meldefonds with RFS Austria
oekb_melde_all = set(oekb_melde_file.loc[oekb_melde_file['ISIN'].str[:2]!='AT']['ISIN'])

Filter generated report file to exclude Funds domiciled in Austria, Umbrella Funds and drop Funds without ISIN (convenience share classes).

In [118]:
criteria_1 = report_file['Domicile']!='Austria'
criteria_2 = report_file['Umbrella Flag']=='No'

criteria_all = criteria_1 & criteria_2

report_file_filtered = report_file[criteria_all].dropna(subset=['ISIN'])

Create sets of __all Funds__ in Report, __all Funds with RFS Austria__ and __all Meldefonds__.

In [119]:
# Lipper all Funds
lipper_all = set(report_file_filtered['ISIN'])

In [120]:
# Lipper Funds with RFS Austria
lipper_rfs = set(report_file_filtered[report_file_filtered['Notified for Sale in Austria']=='Yes']['ISIN'])

In [121]:
melde_criteria_1 = report_file_filtered['Austrian Registration Status']=='Meldefonds'
melde_criteria_2 = report_file_filtered['Austrian Registration Status']=='Meldefonds / Zugelassen'
melde_criteria_3 = report_file_filtered['Austrian Registration Status']=='Zugelassen'
melde_criteria_all = melde_criteria_1 | melde_criteria_2 | melde_criteria_3

# Lipper Meldefonds and Meldefonds with RFS Austria
lipper_melde_all = set(report_file_filtered[melde_criteria_all]['ISIN'])

In [122]:
lipper_melde = set(report_file_filtered[melde_criteria_1]['ISIN'])
lipper_melde_zugelassen = set(report_file_filtered[melde_criteria_2]['ISIN'])
lipper_zugelassen = set(report_file_filtered[melde_criteria_3]['ISIN'])

---

In [123]:
oekb_melde = oekb_melde_all - oekb_rfs
oekb_zugelassen = oekb_rfs - oekb_melde_all
oekb_melde_zugelassen = oekb_melde_all & oekb_rfs

In [124]:
melde_to_delete = lipper_melde_all - (oekb_melde_all | oekb_rfs)

In [125]:
melde_to_add = ((oekb_melde_all | oekb_rfs) - lipper_melde_all) & lipper_all

In [126]:
rfs_to_delete = lipper_rfs - oekb_rfs

In [127]:
rfs_to_add = (oekb_rfs - lipper_rfs) & lipper_all

In [128]:
def filter_melde(x):
    if (x in oekb_melde and x in lipper_melde):
        return False
    if (x in oekb_zugelassen and x in lipper_zugelassen):
        return False
    if(x in oekb_melde_zugelassen and x in lipper_melde_zugelassen):
        return False
    return True

In [129]:
melde_to_filter = ((oekb_melde | oekb_melde_zugelassen | oekb_zugelassen) & lipper_all) - melde_to_add
melde_to_change = set(filter(filter_melde, melde_to_filter))

---

In [130]:
def melde_status(x):
    if (x in oekb_melde):
        return 'Meldefonds'
    if (x in oekb_zugelassen):
        return 'Zugelassen'
    if (x in oekb_melde_zugelassen):
        return 'Meldefonds / Zugelassen'

In [131]:
df_rfs_to_add = pd.DataFrame()
df_rfs_to_add['ISIN Code (Xref)'] = pd.Series(list(rfs_to_add))
df_rfs_to_add['Registered For Sale Country (Add RFS)'] = 'Austria'

In [132]:
df_rfs_to_delete = pd.DataFrame()
df_rfs_to_delete['ISIN Code (Xref)'] = pd.Series(list(rfs_to_delete))
df_rfs_to_delete['Registered For Sale Country (Remove RFS)'] = 'Austria'

In [133]:
df1 = pd.DataFrame()
df1['ISIN Code (Xref)'] = pd.Series(list(melde_to_delete))
df1['Austrian Registration Status'] = 'NULL'
df1['Attribute Start Date'] = '01/01/1800'
df1['Attribute Overwrite Historical Flag'] = 'Yes'

In [134]:
df2 = pd.DataFrame()
df2['ISIN Code (Xref)'] = pd.Series(list(melde_to_change | melde_to_add))
df2['Austrian Registration Status'] = df2['ISIN Code (Xref)'].apply(melde_status)
df2['Attribute Start Date'] = '01/01/1800'
df2['Attribute Overwrite Historical Flag'] = 'Yes'

In [135]:
df_status = pd.concat([df1, df2])

In [136]:
time = datetime.now().strftime("%Y%m%d_%H%M%S")
dirname = '/output/{}'.format(time)
if not os.path.exists(os.getcwd() + dirname):
    try:
        os.makedirs(os.getcwd() + dirname)
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [137]:
df_rfs_to_add.to_csv('output/{0}/RFS_TO_ADD_{0}.csv'.format(time))
df_rfs_to_delete.to_csv('output/{0}/RFS_TO_REMOVE_{0}.csv'.format(time))
df_status.to_csv('output/{0}/STATUS_TO_UPDATE_{0}.csv'.format(time))

---

In [138]:
log_message = """RFS deleted:\t\t{}
RFS added:\t\t{}
Status deleted:\t\t{}
Status added:\t\t{}
Status updated: \t{}
""".format(len(rfs_to_delete), len(rfs_to_add), len(melde_to_delete), len(melde_to_add), len(melde_to_change))

In [139]:
with open("log.txt", "a") as f:
    f.write('-' * 10)
    f.write('\n')
    f.write('-' * 10)
    f.write('\n')
    f.write(str(datetime.now()))
    f.write('\n\n')
    f.write(log_message)
    f.write('\n')